<h1><center> Visual Speech Recognition - Preprocessing </center></h1>

## 📦 Importing Libraries

In [ ]:
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 28.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [ ]:
import os
import cv2
import tensorflow as tf
import numpy as np
import imageio
import gdown
import glob
from google.colab import files

## 🗃️ Data Setup


In [ ]:
#הורדת הדאטה הראשונית שמכילה רק דובר אחד
url = 'https://drive.google.com/uc?id=19fi9UaWCdJnfvcQwklteyHeV8HlDSkpg'
output = 'data.zip'
gdown.download(url, output, quiet=False)

In [ ]:
#הורדת דאטת הואלידציה
url = 'https://drive.google.com/uc?id=1-6KDKqhPwLodMZ7BgesUTH0kNpeZc2Um'
output = 'val.zip'
gdown.download(url, output, quiet=False)

In [ ]:
#הורדת הדאטה הסופית שמכילה שישה דוברים
url = 'https://drive.google.com/uc?id=1CvYlsHBhuUhByBXmGvEJnVKx2dp8hUsa'
output = 'new_data.zip'
gdown.download(url, output, quiet=False)

## ⚙️ Data Pre-processing pt. 1


<div dir="rtl">

מייבא את הספרייה dlib ומוריד את הקובץ "shape_predictor_68_face_landmarks.dat", שמשמש עבור זיהוי הפנים ובפרט השפתיים.
- **`detector`** - מזהה פרצופים בתמונה.  
- **`predictor`** - מזהה 68  landmarks על הפנים, כמו קווי המתאר של העיניים, האף והפה.



</div>


In [ ]:
import dlib
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 shape_predictor_68_face_landmarks.dat.bz2

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

--2025-04-02 00:42:34--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 [following]
--2025-04-02 00:42:34--  https://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Connecting to dlib.net (dlib.net)|107.180.26.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  39.8MB/s    in 1.5s    

2025-04-02 00:42:36 (39.8 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
def load_video(path, width, height):
    cap = cv2.VideoCapture(path)
    frames = []
    for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        ret, frame = cap.read()
        frame = tf.image.rgb_to_grayscale(frame)
        faces = detector(frame.numpy())
        try:
            landmarks = predictor(frame.numpy(), faces[0])

            cx = (landmarks.part(48).x + landmarks.part(54).x) // 2
            cy = (landmarks.part(48).y + landmarks.part(54).y) // 2

            half_width = width // 2
            half_height = height // 2

            if (cy + half_height) > frame.shape[0]:
                cy =  frame.shape[0] - half_height

            left = cx - half_width
            top = cy - half_height
            right = cx + half_width
            bottom = cy + half_height
            frames.append(frame[top:bottom,left:right,:])

        except:
            continue


    cap.release()

    mean = tf.math.reduce_mean(frames)
    std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    norm_frames =  tf.cast((frames - mean), tf.float32) / std
    return norm_frames

In [ ]:
def  preprocess_and_save_videos(input_path, output_dir, width, height, zip_output):

    video_paths = glob.glob(input_path)
    os.makedirs(output_dir, exist_ok=True)

    for path in video_paths:
        file_name = os.path.splitext(os.path.basename(path))[0]
        save_path = os.path.join(output_dir, f'{file_name}.npy')
        frames = load_video(path, width=width, height=height).numpy()
        np.save(save_path, frames)
        print(f"Saved {save_path}")

    if zip_output:
        zip_name = output_dir.rstrip('/').split('/')[-1] + '.zip'
        !zip -r {zip_name} {output_dir}
        print(f"Zipped to {zip_name}")
        files.download(zip_name)


עבור שני המודלים במחברת modelsProofOfConcept

In [ ]:
preprocess_and_save_videos(
    input_path='./data/s1/*.mpg',
    output_dir='./processed_100x50',
    width=100,
    height=50,
    zip_output=True,
)


"\npreprocess_and_save_videos(\n    input_path='./data/s1/*.mpg',\n    output_dir='./processed_100x50',\n    width=100,\n    height=50,\n    zip_output=True,\n    download_zip=True\n)\n"

In [ ]:
preprocess_and_save_videos(
    input_path='./data/s1/*.mpg',
    output_dir='./processed_70x40',
    width=70,
    height=40,
    zip_output=True,
)

עבור אבלואציה של המודלים, עיבוד הדאטה של דובר חדש לגמרי

In [ ]:
preprocess_and_save_videos(
    input_path='./val/s3/*.mpg',
    output_dir='./val_70x40',
    width=70,
    height=40,
    zip_output=True,
)

In [ ]:
preprocess_and_save_videos(
    input_path='./val/s3/*.mpg',
    output_dir='./val_100x50',
    width=100,
    height=50,
    zip_output=True,
)

עיבוד הדאטה הסופית שמורכבת משישה דוברים

In [ ]:
preprocess_and_save_videos(
    input_path='./new_data/s/*.mpg',
    output_dir='./processed_new70X40',
    width=70,
    height=40,
    zip_output=True,
)